---

# 230920
## 목표
1. 230912_MILP_WTA의 데이터 받아서 실행하기
    - 데이터 전처리
    - 제한 함수 삭제
    - 목적함수 수정

In [10]:
import numpy as np
import time

path = "/ssd2/kawon/230727_WTA/data/wta_100x200x1.txt"

weight=[]



# reading data from text file
f = open(path, "r")
lines = f.readlines()
f.close()



# reading first line
W, T, FN_T = [int(v) for v in lines[0].split()]
# asssuming all weapons have the same availability:
FN_T = [FN_T] * W

M = sum(FN_T)


# 0과 99 사이의 실수로 PK_wt 배열 초기화
PK_wt = np.random.uniform(0, 99, (W, T))


# reading target weights
for line in lines[1:T + 1]:
    weight.append(int(line))



# reading probabilities
for line in lines[T + 1:]:
    splitted = line.split()
    w, t, p = int(splitted[0]), int(splitted[1]), float(splitted[2])
    PK_wt[w][t] = p

In [2]:
# M을 W로 나누어 리스트에 적절히 분배
quotient, remainder = divmod(M, W)
m_w = [quotient] * W

# 나머지 값을 리스트의 앞부분에 추가
for i in range(remainder):
    m_w[i] += 1

In [11]:
import copy
# Initialize a 3D binary matrix of zeros
theta_wt = np.zeros((W, T, M), dtype=int)

# 미사일이 소모되는 리스트 left_FN_T으로 복사
left_FN_T = copy.copy(FN_T)

# 발사대에 적재된 유도탄 수량 리스트 left_m_w로 복사
left_m_w = copy.copy(m_w)

# 행렬의 원소들을 1차원 배열로 변환
flattened = (PK_wt*weight).flatten()

# 원소들을 내림차순으로 정렬
sorted_indices = np.argsort(flattened)[::-1]

In [12]:
flattened

array([17.41110853,  3.12795766,  6.0860326 , ...,  4.3433639 ,
       47.75786389,  2.81271673])

In [13]:
i = 0
m = 0
indices_list = []

while True :
    if i == T * W :
        break
    # i번째 요격확률의 인덱스 v
    v = sorted_indices[i]
    
    row = v // T # Weapon
    col = v % T # Target
    print(f"Value: {flattened[v]}, Index: (Weapon : {row}, Target : {col})")
    
    # 해당 Target의 할당할 수 있는 유도탄의 개수가 남아 있다면,
    # 해당 Weapon의 적재된 유도탄이 남아 있다면,
    # if left_FN_T[col] > 0 and left_m_w[row] > 0:
    if left_m_w[row] > 0:
            
        
            
        # 할당할 유도탄 개수 감소
        # left_FN_T[col] -= 1
        left_m_w[row] -= 1
        # 해당 W T M에 1 부여
        theta_wt[row][col][m] = 1
        # 무기가 할당된 indices_list에 추가
        indices_list.append(v)

        # 다음 미사일 인덱스로 이동
        m += 1

    # 해당 Target의 할당할 수 있는 유도탄의 개수가 남아 있지 않다면,
    # elif left_FN_T[col] > 0:
    #     pass
        # print(f"not enough Missile on Weapon {row}")
    
    else :
        pass
        # print(f"not enough Missile for Target {col}")
    
    # 할당할 수 있는 유도탄의 개수를 모두 소모했다면 종료.
    if sum(left_FN_T) == 0:
        break
    

    # 다음으로 요격확률이 높은 인덱스로 이동
    i += 1

Value: 99.1425434177, Index: (Weapon : 61, Target : 41)
Value: 99.0622679884, Index: (Weapon : 78, Target : 41)
Value: 98.5904579827, Index: (Weapon : 65, Target : 107)
Value: 98.555200016328, Index: (Weapon : 6, Target : 100)
Value: 98.5292511815, Index: (Weapon : 85, Target : 41)
Value: 98.3537585478, Index: (Weapon : 90, Target : 41)
Value: 98.217761592, Index: (Weapon : 28, Target : 41)
Value: 97.991260426346, Index: (Weapon : 56, Target : 164)
Value: 97.730170564866, Index: (Weapon : 58, Target : 50)
Value: 97.3182580436, Index: (Weapon : 24, Target : 107)
Value: 97.157868206751, Index: (Weapon : 56, Target : 100)
Value: 96.963460134338, Index: (Weapon : 7, Target : 183)
Value: 96.7883009774, Index: (Weapon : 45, Target : 41)
Value: 96.755999594576, Index: (Weapon : 7, Target : 164)
Value: 96.68675604180599, Index: (Weapon : 3, Target : 50)
Value: 96.56576675360999, Index: (Weapon : 96, Target : 100)
Value: 96.448550486078, Index: (Weapon : 72, Target : 97)
Value: 96.231199691334,

In [14]:
"""
목표함수 (1)
해당 목적함수는 결정변수 theta_wt와 요격확률의 곱의 합을 최대화함으로써 요격확률이 높을 때 표적-발사대 쌍의 발사 시점에 무기할당을 하는 의미로 해석할 수 있다.
"""
def objective_fun_1(theta_wt, weight):
    res = 0
    for t in range(T):
        
        a = 1
        for w in range(W):
            for m in range(M):
                p = 1 - PK_wt[w,t] * theta_wt[w,t,m]
                a *= p
        
        a *= weight[t]

        print(f"타깃 {t}에 대한 성공 값 : {a}")
        res += a


    return res

In [15]:
objective_fun_1(theta_wt,weight)

타깃 0에 대한 성공 값 : 54.0
타깃 1에 대한 성공 값 : 49.0
타깃 2에 대한 성공 값 : 66.0
타깃 3에 대한 성공 값 : 61.0
타깃 4에 대한 성공 값 : 8.655948611813818e-06
타깃 5에 대한 성공 값 : 29.0
타깃 6에 대한 성공 값 : 51.0
타깃 7에 대한 성공 값 : 65.0
타깃 8에 대한 성공 값 : 5.0
타깃 9에 대한 성공 값 : 21.0
타깃 10에 대한 성공 값 : 51.0
타깃 11에 대한 성공 값 : 63.0
타깃 12에 대한 성공 값 : 50.0
타깃 13에 대한 성공 값 : 42.0
타깃 14에 대한 성공 값 : 66.0
타깃 15에 대한 성공 값 : 22.0
타깃 16에 대한 성공 값 : 59.0
타깃 17에 대한 성공 값 : 30.0
타깃 18에 대한 성공 값 : 17.0
타깃 19에 대한 성공 값 : 13.0
타깃 20에 대한 성공 값 : 55.0
타깃 21에 대한 성공 값 : 19.0
타깃 22에 대한 성공 값 : 2.4380931135815398e-05
타깃 23에 대한 성공 값 : 17.0
타깃 24에 대한 성공 값 : 0.20573570194432356
타깃 25에 대한 성공 값 : 25.0
타깃 26에 대한 성공 값 : 15.0
타깃 27에 대한 성공 값 : 33.0
타깃 28에 대한 성공 값 : 54.0
타깃 29에 대한 성공 값 : 42.0
타깃 30에 대한 성공 값 : 0.10927791679982066
타깃 31에 대한 성공 값 : 87.0
타깃 32에 대한 성공 값 : 30.0
타깃 33에 대한 성공 값 : 34.0
타깃 34에 대한 성공 값 : 30.0
타깃 35에 대한 성공 값 : 34.0
타깃 36에 대한 성공 값 : 56.0
타깃 37에 대한 성공 값 : 4.5673372392405815e-08
타깃 38에 대한 성공 값 : 78.0
타깃 39에 대한 성공 값 : 58.0
타깃 40에 대한 성공 값 : 38.0
타깃 41에 대한 성공 값 : 3.2427970

7772.660068136787

In [43]:
# 0이 아닌 원소의 인덱스 찾기
non_zero_indices = np.argwhere(theta_wt != 0)

# 인덱스 출력
for index in non_zero_indices:
    temp_w, temp_t, temp_m = tuple(index)
    print(f"x({temp_w},{temp_t})={theta_wt[temp_w][temp_t][temp_m]}")

x(0,37)=1
x(1,95)=1
x(2,19)=1
x(3,87)=1
x(4,43)=1
x(5,55)=1
x(6,71)=1
x(7,58)=1
x(8,92)=1
x(9,66)=1
x(10,47)=1
x(11,99)=1
x(12,27)=1
x(13,62)=1
x(14,79)=1
x(15,4)=1
x(16,78)=1
x(17,96)=1
x(18,0)=1
x(19,73)=1
x(20,49)=1
x(21,63)=1
x(22,17)=1
x(23,69)=1
x(24,51)=1
x(25,91)=1
x(26,1)=1
x(27,48)=1
x(28,23)=1
x(29,38)=1
x(30,42)=1
x(31,20)=1
x(32,21)=1
x(33,10)=1
x(34,9)=1
x(35,54)=1
x(36,46)=1
x(37,72)=1
x(38,50)=1
x(39,98)=1
x(40,29)=1
x(41,3)=1
x(42,84)=1
x(43,35)=1
x(44,56)=1
x(45,26)=1
x(46,60)=1
x(47,70)=1
x(48,65)=1
x(49,81)=1


# for runtime

In [41]:

import numpy as np
import time

start_time = time.time()

path = "/ssd2/kawon/230727_WTA/data/wta_50x100x1.txt"

weight=[]



# reading data from text file
f = open(path, "r")
lines = f.readlines()
f.close()



# reading first line
W, T, x = [int(v) for v in lines[0].split()]
# asssuming all weapons have the same availability:
m_w = [x] * W

M = sum(m_w)

FN_T = [x] * T

# 0과 99 사이의 실수로 PK_wt 배열 초기화
PK_wt = np.zeros((W, T))


# reading target weights
for line in lines[1:T + 1]:
    weight.append(int(line))



# reading probabilities
for line in lines[T + 1:]:
    splitted = line.split()
    w, t, p = int(splitted[0]), int(splitted[1]), float(splitted[2])
    PK_wt[w][t] = p


# M을 W로 나누어 리스트에 적절히 분배
quotient, remainder = divmod(M, W)
m_w = [quotient] * W

# 나머지 값을 리스트의 앞부분에 추가
for i in range(remainder):
    m_w[i] += 1

import copy
# Initialize a 3D binary matrix of zeros
theta_wt = np.zeros((W, T, M), dtype=int)


# 발사대에 적재된 유도탄 수량 리스트 left_m_w로 복사
left_m_w = copy.copy(m_w)

left_FN_T = copy.copy(FN_T)

# 행렬의 원소들을 1차원 배열로 변환
flattened = PK_wt.flatten()

# 원소들을 내림차순으로 정렬
sorted_indices = np.argsort(flattened)[::-1]

i = 0
m = 0
indices_list = []

while True :
    if i == T * W :
        break
    # i번째 요격확률의 인덱스 v
    v = sorted_indices[i]
    
    row = v // T # Weapon
    col = v % T # Target
    print(f"Value: {flattened[v]}, Index: (Weapon : {row}, Target : {col})")
    
    # 해당 Target의 할당할 수 있는 유도탄의 개수가 남아 있다면,
    # 해당 Weapon의 적재된 유도탄이 남아 있다면,
    # if left_m_w[row] > 0:
    if left_FN_T[col] > 0 and left_m_w[row] > 0:
            
        
            
        # 할당할 유도탄 개수 감소
        left_FN_T[col] -= 1
        left_m_w[row] -= 1
        # 해당 W T M에 1 부여
        theta_wt[row][col][m] = 1
        # 무기가 할당된 indices_list에 추가
        indices_list.append(v)

        # 다음 미사일 인덱스로 이동
        m += 1

    # 해당 Target의 할당할 수 있는 유도탄의 개수가 남아 있지 않다면,
    elif left_FN_T[col] > 0:
        pass
        print(f"not enough Missile on Weapon {row}")
    
    else :
        pass
        # print(f"not enough Missile for Target {col}")
    
    # 할당할 수 있는 유도탄의 개수를 모두 소모했다면 종료.
    if sum(left_m_w) == 0:
        break
    

    # 다음으로 요격확률이 높은 인덱스로 이동
    i += 1

end_time = time.time()

elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Value: 0.999908333929, Index: (Weapon : 14, Target : 79)
Value: 0.999768899001, Index: (Weapon : 9, Target : 66)
Value: 0.999752754517, Index: (Weapon : 19, Target : 73)
Value: 0.999703408776, Index: (Weapon : 10, Target : 47)
Value: 0.999495375774, Index: (Weapon : 13, Target : 62)
Value: 0.999113550816, Index: (Weapon : 43, Target : 66)
Value: 0.999048291878, Index: (Weapon : 27, Target : 48)
Value: 0.998907563007, Index: (Weapon : 19, Target : 14)
not enough Missile on Weapon 19
Value: 0.998892149437, Index: (Weapon : 12, Target : 27)
Value: 0.998769585667, Index: (Weapon : 26, Target : 1)
Value: 0.998462391648, Index: (Weapon : 25, Target : 91)
Value: 0.998405988793, Index: (Weapon : 37, Target : 72)
Value: 0.998320847532, Index: (Weapon : 32, Target : 48)
Value: 0.998061066825, Index: (Weapon : 35, Target : 1)
Value: 0.997906576074, Index: (Weapon : 26, Target : 63)
not enough Missile on Weapon 26
Value: 0.997067189233, Index: (Weapon : 9, Target : 33)
not enough Missile on Weapon

In [42]:
objective_fun_1(theta_wt,weight)

타깃 0에 대한 성공 값 : 0.12584098365800056
타깃 1에 대한 성공 값 : 0.07382485997999844
타깃 2에 대한 성공 값 : 71.0
타깃 3에 대한 성공 값 : 0.6291455712060038
타깃 4에 대한 성공 값 : 0.1555734239640003
타깃 5에 대한 성공 값 : 46.0
타깃 6에 대한 성공 값 : 82.0
타깃 7에 대한 성공 값 : 82.0
타깃 8에 대한 성공 값 : 43.0
타깃 9에 대한 성공 값 : 0.010782058393999927
타깃 10에 대한 성공 값 : 2.6365579190459982
타깃 11에 대한 성공 값 : 60.0
타깃 12에 대한 성공 값 : 19.0
타깃 13에 대한 성공 값 : 91.0
타깃 14에 대한 성공 값 : 66.0
타깃 15에 대한 성공 값 : 23.0
타깃 16에 대한 성공 값 : 95.0
타깃 17에 대한 성공 값 : 2.029105122360001
타깃 18에 대한 성공 값 : 56.0
타깃 19에 대한 성공 값 : 1.017430051624002
타깃 20에 대한 성공 값 : 0.31583268730300007
타깃 21에 대한 성공 값 : 3.1201993530660017
타깃 22에 대한 성공 값 : 28.0
타깃 23에 대한 성공 값 : 2.2692206781300044
타깃 24에 대한 성공 값 : 64.0
타깃 25에 대한 성공 값 : 91.0
타깃 26에 대한 성공 값 : 0.4889252960199997
타깃 27에 대한 성공 값 : 0.06536318321700008
타깃 28에 대한 성공 값 : 61.0
타깃 29에 대한 성공 값 : 0.42942911489999847
타깃 30에 대한 성공 값 : 42.0
타깃 31에 대한 성공 값 : 30.0
타깃 32에 대한 성공 값 : 60.0
타깃 33에 대한 성공 값 : 96.0
타깃 34에 대한 성공 값 : 85.0
타깃 35에 대한 성공 값 : 0.8329616010570022
타깃 

2917.6152504590364

Target : 0, res: 0.00

Elapsed time: 0.0113525390625 seconds

Target : 100, res: 94.64

Elapsed time: 6.394922733306885 seconds

Target : 200, res: 193.84

Elapsed time: 23.664151906967163 seconds

Target : 300, res: 292.16

Elapsed time: 52.60361385345459 seconds

Target : 400, res: 390.56

Elapsed time: 93.8756833076477 seconds

Target : 500, res: 489.18

Elapsed time: 147.16718006134033 seconds

Target : 600, res: 588.93

Elapsed time: 210.77380347251892 seconds

Target : 700, res: 688.24

Elapsed time: 286.8595986366272 seconds

Target : 800, res: 786.17

Elapsed time: 374.906601190567 seconds

Target : 900, res: 884.81

Elapsed time: 475.8631122112274 seconds

Target : 1000, res: 984.37

Elapsed time: 587.348771572113 seconds

Target : 1100, res: 1084.19

Elapsed time: 737.4572696685791 seconds

Target : 1200, res: 1182.09

Elapsed time: 843.8965196609497 seconds

Target : 1300, res: 1280.86

Elapsed time: 994.9845380783081 seconds

Target : 1400, res: 1379.28

Elapsed time: 1156.028076171875 seconds

Target : 1500, res: 1478.83

Elapsed time: 1324.567284822464 seconds

**WEAPON 1000, TARGET (0:1600:100) 결과 (SEED(1))**

In [11]:
import numpy as np

# reading first line
W, T, mu = [int(v) for v in lines[0].split()]
# asssuming all weapons have the same availability:
FN_T = [mu] * T

w = []

M = mu * T

PK_wt = np.zeros((W, T), dtype=float)

# reading target weights
for line in lines[1:T + 1]:
    w.append(int(line))



# reading probabilities
for line in (lines[T + 1:]):
    splitted = line.split()
    i, j, p = int(splitted[0]), int(splitted[1]), float(splitted[2])
    PK_wt[i][j] = p

In [11]:
# M을 W로 나누어 리스트에 적절히 분배
quotient, remainder = divmod(M, W)
m_w = [quotient] * W

# 나머지 값을 리스트의 앞부분에 추가
for i in range(remainder):
    m_w[i] += 1

import copy
# Initialize a 3D binary matrix of zeros
theta_wt = np.zeros((W, T, M), dtype=int)

# 미사일이 소모되는 리스트 left_FN_T으로 복사
left_FN_T = copy.copy(FN_T)

# 발사대에 적재된 유도탄 수량 리스트 left_m_w로 복사
left_m_w = copy.copy(m_w)

# 행렬의 원소들을 1차원 배열로 변환
flattened = PK_wt.flatten()

# 원소들을 내림차순으로 정렬
sorted_indices = np.argsort(flattened)[::-1]


i = 0
m = 0
indices_list = []
while True :
    if i == T * W :
        break
    # i번째 요격확률의 인덱스 v
    v = sorted_indices[i]
    
    row = v // T # Weapon
    col = v % T # Target
    print(f"Value: {flattened[v]}, Index: (Weapon : {row}, Target : {col})")
    
    # 해당 Target의 할당할 수 있는 유도탄의 개수가 남아 있다면,
    # 해당 Weapon의 적재된 유도탄이 남아 있다면,
    if left_FN_T[col] > 0 and left_m_w[row] > 0:
        
        

            
        
             
        # 할당할 유도탄 개수 감소
        left_FN_T[col] -= 1
        left_m_w[row] -= 1
        # 해당 W T M에 1 부여
        theta_wt[row][col][m] = 1
        # 무기가 할당된 indices_list에 추가
        indices_list.append(v)

        # 다음 미사일 인덱스로 이동
        m += 1

    # 해당 Target의 할당할 수 있는 유도탄의 개수가 남아 있지 않다면,
    elif left_FN_T[col] > 0:
        print(f"not enough Missile on Weapon {row}")
    
    else :
        print(f"not enough Missile for Target {col}")
    
    # 할당할 수 있는 유도탄의 개수를 모두 소모했다면 종료.
    if sum(left_FN_T) == 0:
        break
    

    # 다음으로 요격확률이 높은 인덱스로 이동
    i += 1


Value: 0.999908333929, Index: (Weapon : 14, Target : 79)
Value: 0.999768899001, Index: (Weapon : 9, Target : 66)
Value: 0.999752754517, Index: (Weapon : 19, Target : 73)
Value: 0.999703408776, Index: (Weapon : 10, Target : 47)
Value: 0.999495375774, Index: (Weapon : 13, Target : 62)
Value: 0.999113550816, Index: (Weapon : 43, Target : 66)
not enough Missile for Target 66
Value: 0.999048291878, Index: (Weapon : 27, Target : 48)
Value: 0.998907563007, Index: (Weapon : 19, Target : 14)
Value: 0.998892149437, Index: (Weapon : 12, Target : 27)
Value: 0.998769585667, Index: (Weapon : 26, Target : 1)
Value: 0.998462391648, Index: (Weapon : 25, Target : 91)
Value: 0.998405988793, Index: (Weapon : 37, Target : 72)
Value: 0.998320847532, Index: (Weapon : 32, Target : 48)
not enough Missile for Target 48
Value: 0.998061066825, Index: (Weapon : 35, Target : 1)
not enough Missile for Target 1
Value: 0.997906576074, Index: (Weapon : 26, Target : 63)
Value: 0.997067189233, Index: (Weapon : 9, Target 

In [12]:
def simulation(theta_wt):
    W, T, M = theta_wt.shape
    result = []
    for m in range(M):
        indices = np.where(theta_wt[:,:,m] == 1)
        if len(indices[0]) == 0 :
            continue
        w = int(indices[0])
        t = int(indices[1])



        PK = PK_wt[w][t]
        result.append( [w, t, m, PK])

    if constraint2(theta_wt):
        print("제약조건 2 충족")
    if constraint3(theta_wt):
        print("제약조건 3 충족")
    # if constraint4(tau):
    #     print("제약조건 4 충족")
    # if constraint5(tau, theta_wt):
        print("제약조건 5 충족")
    if constraint6(theta_wt):
        print("제약조건 6 충족")

    result.sort()
    for res in result:

        print(f"발사대 {res[0]}, 타깃 {res[1]}을 향해 미사일 {res[2]} 발사, 요격 확률 : {res[3]:.2f}")

    objective_value = objective_fun_1(theta_wt)

    print(f"목표함수 값 : {objective_value}")


In [13]:
simulation(theta_wt)

제약조건 2 충족
제약조건 3 충족
제약조건 5 충족
제약조건 6 충족
발사대 0, 타깃 31을 향해 미사일 57 발사, 요격 확률 : 0.97
발사대 0, 타깃 37을 향해 미사일 31 발사, 요격 확률 : 0.99
발사대 1, 타깃 12을 향해 미사일 74 발사, 요격 확률 : 0.96
발사대 1, 타깃 95을 향해 미사일 20 발사, 요격 확률 : 1.00
발사대 2, 타깃 19을 향해 미사일 51 발사, 요격 확률 : 0.98
발사대 2, 타깃 65을 향해 미사일 64 발사, 요격 확률 : 0.97
발사대 3, 타깃 39을 향해 미사일 78 발사, 요격 확률 : 0.95
발사대 3, 타깃 87을 향해 미사일 43 발사, 요격 확률 : 0.99
발사대 4, 타깃 43을 향해 미사일 23 발사, 요격 확률 : 0.99
발사대 4, 타깃 86을 향해 미사일 36 발사, 요격 확률 : 0.99
발사대 5, 타깃 55을 향해 미사일 67 발사, 요격 확률 : 0.97
발사대 5, 타깃 64을 향해 미사일 97 발사, 요격 확률 : 0.69
발사대 6, 타깃 22을 향해 미사일 92 발사, 요격 확률 : 0.86
발사대 6, 타깃 45을 향해 미사일 83 발사, 요격 확률 : 0.92
발사대 7, 타깃 7을 향해 미사일 66 발사, 요격 확률 : 0.97
발사대 7, 타깃 58을 향해 미사일 17 발사, 요격 확률 : 1.00
발사대 8, 타깃 90을 향해 미사일 76 발사, 요격 확률 : 0.96
발사대 8, 타깃 92을 향해 미사일 54 발사, 요격 확률 : 0.98
발사대 9, 타깃 33을 향해 미사일 12 발사, 요격 확률 : 1.00
발사대 9, 타깃 66을 향해 미사일 1 발사, 요격 확률 : 1.00
발사대 10, 타깃 47을 향해 미사일 3 발사, 요격 확률 : 1.00
발사대 10, 타깃 54을 향해 미사일 25 발사, 요격 확률 : 0.99
발사대 11, 타깃 32을 향해 미사일 85 발사, 요격 확률 : 0.92
발사대 11, 타깃 99을 향해